# Jupyter widget to label the imaging data
Some images are randomized (`CTF-P006` project) - even if image has status `OK` the ordering of wells is random. However, we don't have an access to the project name in our data and will rely on visual filtering out those images because they break the pattern. Those images should first be de-randomized first, when the mapping will be available. Meanwhile, we just exclude those images from consideration.<br>

Based on the [labeling tool](https://github.com/dida-do/public/tree/master/labelingtool) repo.

In [30]:
# add parent dir to path
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
import numpy as np
from pathlib import Path

from IPython.display import Image, clear_output, display
from ipywidgets import Button, HBox, Output, Layout
from tqdm.notebook import tqdm
from matplotlib.image import imsave

In [31]:
from params import PATH_OUT
path_npy = PATH_OUT / 'data_classifier' # path to .npy files

Generate 10 times larger images (replace each pixel by `10x10` pixels):

In [32]:
path_ims = PATH_OUT / 'data_classifier_png_large'
path_ims.mkdir(exist_ok=True)

In [33]:
for f in tqdm(path_npy.glob('*.npy')):
    arr = np.load(f)
    arr = arr.repeat(10, axis=0).repeat(10, axis=1)
    name = f.name.split('.')[0]
    imsave(path_ims / f"{name}.png", arr)

0it [00:00, ?it/s]

Define class for imaage labelling:

In [37]:
"""Simple tool for class labeling of images in Jupyter notebooks.

Classes:
    ImageClassLabeler

"""

class LabelingTool:
    """A tool to be used for image classification labeling.

    Needs to run in a jupyter notebook to display ipywidgets.

    Attributes:
        classes: A list of image classes to be assigned during labeling.
        path: The path to the directory that contains the images.
        images: A list of the image files in the directory.
        current: Pointer to the current position of the tool.
        labels: Dictionary with assigned labels {'filename': 'label'}.
    """

    def __init__(
        self,
        classes: list,
        path: str,
    ) -> None:
        """Construct all necessary attributes for the LabelingTool object.

        Args:
            classes: List of candidate labels (typically strings).
            path: Path to the directory containing the images to be labeled.

        """
        # store list of available class labels
        self.classes = classes

        # store path and list of images in path
        self.path = Path(path)
        self.images = [f.name for f in self.path.glob("*.png")]

        # set up empty dict for labels and initial position
        self.labels = {}
        self.position = 0

    def _next(self, *args) -> None:
        """Select the next image and updates the displays.

        Args:
            *args: Variable length argument list, allows to pass Button object.

        """
        # update position
        self.position += 1
        if self.position == len(self.images):
            self.position = 0

        # refresh display
        image = Image(self.path / self.images[self.position])
        with self.frame:
            clear_output(wait=True)
            display(image)
        

    def _go_back(self, *args) -> None:
        """Select the previous image and updates the displays.

        Do not do anything if it is the first image.

        Args:
            *args: Variable length argument list, allows to pass Button object.

        """
        # update position
        self.position -= 1
        if self.position == -1:
            self.position = len(self.images) - 1

        # refresh display
        image = Image(self.path / self.images[self.position])
        with self.frame:
            clear_output(wait=True)
            display(image)
        

    def _select_label(self, button: Button) -> None:
        """Attach a label to the current image and stores it.

        Link the label from a given button to the current image and store it
        in the labels dictionary and in the current working directory as .json.

        Args:
            button (ipywidgets.Button): a button supplying the label to be
                associated with the current image

        """
        # store label
        current_image = self.images[self.position]
        self.labels[current_image] = button.description

        # move on
        self._next()

    def start(self) -> None:
        """Start the labeling procedure.

        Load the first image to label and set up the user interface.

        """
        # image frame
        image = Image(self.path / self.images[self.position])
        self.frame = Output(layout=Layout(height="160px", max_width="240px"))
        with self.frame:
            display(image)
        
        
        # navigation buttons
        backward_button = Button(description="< go back")
        backward_button.on_click(self._go_back)
        forward_button = Button(description="next >")
        forward_button.on_click(self._next)
        self.navigation_buttons = [backward_button, forward_button]

        # class label buttons
        self.class_buttons = []
        for label in self.classes:
            label_button = Button(description=label)
            label_button.on_click(self._select_label)
            self.class_buttons.append(label_button)

        # display contents
        display(self.frame)
        display(HBox(self.navigation_buttons))
        display(HBox(self.class_buttons))

In [38]:
tool = LabelingTool(path=path_ims, classes=["good", "randomized"])

In [39]:
tool.start()

Output(layout=Layout(height='160px', max_width='240px'))

In [40]:
print(tool.labels)

{}
